## Transformación y Carga de Datos

### Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

### Lectura de datos con Pandas

In [2]:
df = pd.read_csv(r'../data/data.csv', sep=',', encoding='latin-1')

### Procesamiento de datos

In [3]:
#Verificador de dimension del DataFrame (número de filas y columnas)
df.shape

(7663, 96)

In [4]:
#Ver una muestra de los datos
df.head()

,Unnamed: 0.1,Unnamed: 0,Antiguedad,Anunciante,Balneario,NroBanios,Nro_pisos,Cocheras,Descripcion,Direccion,...,'Aire acondicionado','Amoblado','Sauna','Calefaccion','Equipado','Terma','Portero electrico','Cocina con reposteros',Distrito,Provincia
0,0,3,5.0,Valentinoinmobiliaria,Nulo,4.0,2.0,3.0,excelente opcion para vivir en una de las mej...,"Malecon Monte Bello 5 La Molina, La Planicie, ...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LaMolina,Lima
1,1,4,5.0,No disponible,Nulo,5.0,3.0,1.0,hermosa casa a espalda de futura avenida prol...,Mz. O2 Lt. 03 Urb. San Antonio De Carapongo 2 ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lurigancho,Lima
2,2,5,3.0,No disponible,Nulo,5.0,3.0,4.0,casa de estreno acabados de primera. calle ce...,"El Pinar 120 , La Planicie 1era Etapa La Molin...",...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,LaMolina,Lima
3,5,8,1.0,No disponible,Sol Y Mar,5.0,2.0,3.0,linda casa en condominio con piscina club hou...,"Km 98.5 De La Panamericana Sur Asia, Sol Y Mar...",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Asia,Lima
4,6,9,4.0,No disponible,Nulo,4.0,2.0,2.0,monterrico 2 casas en 1 ( casa bajos 4 dormit...,"Mariano De Rivera Y Ustariz #215 Surco, Huerto...",...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,SantiagoDeSurco,Lima


In [5]:
#Tipo de Dato de las Columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7663 entries, 0 to 7662
Data columns (total 96 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0.1                    7663 non-null   int64  
 1   Unnamed: 0                      7663 non-null   int64  
 2   Antiguedad                      7663 non-null   float64
 3   Anunciante                      7663 non-null   object 
 4   Balneario                       7663 non-null   object 
 5   NroBanios                       7663 non-null   float64
 6   Nro_pisos                       7663 non-null   float64
 7   Cocheras                        7663 non-null   float64
 8   Descripcion                     7663 non-null   object 
 9   Direccion                       7663 non-null   object 
 10  Dormitorios                     7663 non-null   object 
 11  Estado de Inmueble              7663 non-null   object 
 12  Fecha_pub                       76

#### Transformaciones

In [6]:
#Función para extraer la comilla simple presente en los nombres de la columnas
def removeQuotes(text):
    return text.strip("'")

In [7]:
# Aplicar la función remove_quotes a los nombres de la columnas
df.rename(columns=removeQuotes, inplace=True)

##### Eliminación de columnas

In [8]:
#Las siguientes columnas se han eliminado por los siguientes casos: contiene valores no explicativos,
#columna con nombre poco descriptivo, redundancia, irrelevante y valores inconsistentes
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Fecha_pub', 'match','Kitchenette', 'Anunciante','anx81tico'], inplace=True)
df.drop(columns=['Guardiananxada','Jardanxadn','Ubicacion','Direccion','Area_constr_m2','Area_total_m2'], inplace=True)

##### Verificando Duplicados

In [9]:
print(f"Numero de filas duplicadas = {df.duplicated().sum()}")

Numero de filas duplicadas = 18


In [10]:
# Remover registros duplicados
df = df.drop_duplicates()
df.reset_index(drop=True,inplace=True)

##### Normalización de Datos

In [11]:
#Conversión a minúscula los nombre de las columnas del Dataframe
df.columns= df.columns.str.lower()

In [12]:
#Función que extare comillas simples, dobles y convierte los valores de las columnas de tipo object a minúscula
def getNormalize(text):
    text = str.lstrip(str.rstrip(text))
    text = text.replace("'", '')
    text = text.replace('"', '')
    text = str.lower(text)

    return text

In [13]:
#Aplicación de la función getNormalize
df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').apply(lambda x: x.apply(getNormalize))

In [14]:
#Reemplazo de valores para un posterior cambio de tipo de columna object a tipo numérico
df['dormitorios'] = df['dormitorios'].str.replace('+', '')
df['area_constr'] = df['area_constr'].str.replace('m2', '')
df['area_total'] = df['area_total'].str.replace('m2', '')

In [15]:
#Mapeo de columnas donde sus valores hacen referencia a un valor nulo
columnas_actualizar = ['cuarto de servicio', 'deposito', 'terraza','sala de estar', 'sotano', 'patio', 'comedor diario', 'comedor','banio de servicio', 'jardanxadn interno', 'walking closet',
                       'escritorio', 'cocina', 'banio independiente', 'lavanderanxada','balcon', 'sala', 'closet', 'banio de visitas', 'agua', 'internet','luz', 'cable', 'servicio de limpieza',
                       'conexion a gas','sistema de seguridad', 'telefono', 'areadeportiva', 'piscina','solarium', 'sala de internet', 'sala de cine', 'parque interno','guarderanxada', 'anx81reas verdes',
                       'club house', 'hall de ingreso','areade bbq', 'gimnasio', 'areade sauna', 'juegos para ninios','ingreso independiente', 'colegios cercanos', 'cerca al mar', 'centros comerciales cercanos',
                       'parques cercanos','acceso personas discapacidad', 'frente al mar', 'desagaue', 'jacuzzi','chimenea', 'intercomunicador', 'cerco electrico', 'parrilla','aire acondicionado', 'amoblado',
                       'sauna', 'calefaccion', 'equipado', 'terma', 'portero electrico', 'cocina con reposteros']
df[columnas_actualizar] = df[columnas_actualizar].apply(lambda x: x.str.replace('noespecifica', ''))

In [16]:
columnasActualizar2 = ['balneario', 'estado', 'luminosidad', 'tipocochera', 'comercial', 'profesional']

In [17]:
df['balneario'] = df['balneario'].str.replace('nulo', '')

In [18]:
# Cambio del tipo de columna a numérico
df['dormitorios'] = df['dormitorios'].astype('Int64')
df['latitud'] = pd.to_numeric(df['latitud'], errors="coerce")
df['longitud'] = pd.to_numeric(df['longitud'], errors="coerce")
df['area_constr'] = pd.to_numeric(df['area_constr'], errors="coerce")
df['area_total'] = pd.to_numeric(df['area_total'], errors="coerce")


df[columnas_actualizar] = df[columnas_actualizar].apply(pd.to_numeric, errors="coerce").astype('category')

In [19]:
#Renombrar columnas
nuevos_nombres = {'nrobanios': 'baños', 'nro_pisos': 'pisos', 'estado de inmueble': 'estado', 'uso_comercial': 'comercial', 'uso_profesional': 'profesional',  'cuarto de servicio': 'cuarto_servicio',
                  'sala de estar': 'sala_estar', 'comedor diario': 'comedor_diario', 'banio de servicio': 'baño_servicio', 'jardanxadn interno': 'jardin_interno', 'walking closet': 'walking_closet',
                  'banio_independiente': 'baño_independiente', 'lavanderanxada': 'lavanderia', 'banio de visitas': 'baño_visitas','servicio de limpieza': 'servicio_limpieza','conexion a gas': 'gas',
                  'sistema de seguridad': 'sistema_seguridad', 'areadeportiva': 'area_deportiva', 'sala de internet': 'sala_internet', 'sala de cine': 'sala_cine', 'parque interno': 'parque_interno',
                  'guarderanxada': 'guarderia', 'anx81reas verdes': 'areas_verdes', 'club house': 'club_house', 'hall de ingreso': 'hall_ingreso', 'areade bbq': 'area_bbq', 'areade sauna': 'area_sauna',
                  'juegos para ninios': 'juegos_niños', 'ingreso independiente': 'ingreso_independiente', 'colegios cercanos': 'colegio_cerca', 'cerca al mar': 'mar_cerca',
                  'centros comerciales cercanos': 'centro_comercial_cerca', 'parques cercanos': 'parque_cerca', 'acceso personas discapacidad': 'acceso_discapacidad', 'frente al mar': 'frente_mar',
                  'cerco electrico':'cerco_electrico','aire acondicionado':'aire_acondicionado','portero electrico':'portero_electrico','cocina con reposteros':'cocina_amoblada'}
df.rename(columns=nuevos_nombres, inplace=True)

In [20]:
df[columnasActualizar2] = df[columnasActualizar2].apply(lambda x: x.str.replace('noespecifica', ''))

In [21]:
df.profesional.unique()

array(['', 'si', 'no'], dtype=object)

In [22]:
#Indenticación de las columnas que contienen valores 'si', 'no' y ''
columnas_a_transformar = ['mascotas', 'comercial', 'profesional']

#Mapeao de los valores en las columnas  (Utilizamos np.nan para representar valores vacíos)
mapeo = {'si': 1, 'no': 0, '': np.NaN}

# Aplicar el mapeo y convertir a categorico

for col in columnas_a_transformar:
    df[col] = np.select([df[col] == valor for valor in mapeo.keys()], list(mapeo.values()), default=df[col])



#cambio a tipo category
df[columnas_a_transformar] = df[columnas_a_transformar].apply(pd.to_numeric, errors="coerce").astype('category')

In [23]:
# Codificación one-hot
df_estado_dummy = pd.get_dummies(df['estado'], prefix='estado')
df_lumino_dummy = pd.get_dummies(df['luminosidad'], prefix='luminosidad')
df_tipo_dummy = pd.get_dummies(df['tipo'], prefix='tipo')
df_tipocochera_dummy = pd.get_dummies(df['tipocochera'], prefix='tipocochera')

In [24]:
# Convertir los valores booleanos a 0 y 1
df_estado_dummy = df_estado_dummy.astype(int)
df_lumino_dummy = df_lumino_dummy.astype(int)
df_tipo_dummy = df_tipo_dummy.astype(int)
df_tipocochera_dummy = df_tipocochera_dummy.astype(int)

In [25]:
#Concatenando con el Dataframe principal
df = pd.concat([df,df_estado_dummy], axis=1)
df = pd.concat([df,df_lumino_dummy], axis=1)
df = pd.concat([df,df_tipo_dummy], axis=1)
df = pd.concat([df,df_tipocochera_dummy], axis=1)

In [26]:
#Eliminando columnas posrterior a la conversion del tipo de columna
df.drop(columns=['estado'], inplace=True)
df.drop(columns=['luminosidad'], inplace=True)
df.drop(columns=['tipo'], inplace=True)
df.drop(columns=['tipocochera'], inplace=True)

In [27]:
# Transformar la columna 'balneario' y Definir una función para aplicar la regla en dicha columna
def transformValue(valor):
    return 0 if valor == '' else 1

# Aplicar la función a la columna
df['balneario'] = df['balneario'].apply(transformValue)

# Convertir la columna a tipo numérico
#aqui se cambio antes Int64 ahora category
df['balneario'] = pd.to_numeric(df['balneario']).astype('category')

In [28]:
#Renombrar columnas despues de las transformaciones
nuevos_nombres_columnas = {'estado_a remodelar': 'estado_aremodelar', 'estado_muy bueno': 'estado_muy_bueno', 'luminosidad_muy luminoso': 'luminosidad_muy_luminoso', 'luminosidad_poco luminoso': 'luminosidad_poca',
                  'tipo_casa de campo': 'tipo_campo', 'tipo_casa de playa': 'tipo_playa', 'tipo_casa de playa en condominio': 'tipo_playa_condominio',  'tipo_casa en condominio': 'tipo_condominio',
                  'tipo_casa en quinta':'tipo_quinta', 'tipocochera_lineales':'cochera_lineal', 'tipocochera_notiene':'cochera_notiene','tipocochera_paralelas':'cochera_paralela',
                  'tipocochera_separadas':'cochera_separadas','banio independiente':'baño_independiente','desagaue':'desague'
                 }
df.rename(columns=nuevos_nombres_columnas, inplace=True)

In [29]:
df['baños'] = df['baños'].astype('Int64')
df['pisos'] = df['pisos'].astype('Int64')
df['cocheras'] = df['cocheras'].astype('Int64')
df['antiguedad'] = df['antiguedad'].astype('Int64')

In [30]:
#eliminar
df.drop(columns=['estado_'], inplace=True)
df.drop(columns=['luminosidad_'], inplace=True)
df.drop(columns=['tipocochera_'], inplace=True)

In [31]:
#cambiar  tipo
prefijos = ['estado_', 'tipo_', 'luminosidad_', 'cochera_']

# Itera sobre los prefijos y cambia el tipo de las columnas correspondientes
for prefijo in prefijos:
    columnas_seleccionadas = df.filter(regex=f'^{prefijo}', axis=1)
    df[columnas_seleccionadas.columns] = columnas_seleccionadas.astype('category')



In [32]:
#Filtrar por Lima
df = df[df['provincia'].isin(['lima'])]


In [33]:
# Agrupacion de las columnas estado_excelente y estado_muy_bueno para asignar 1 si alguna de las columnas tiene el valor 1
df['estado_excelente'] = df.apply(lambda row: 1 if row['estado_excelente'] == 1 or row['estado_muy_bueno'] == 1 else np.NaN, axis=1)

df.drop(columns=['estado_muy_bueno'], inplace=True)

In [34]:
# Agrupacion de las columnas cochera_lineal, cochera_paralela, cochera_separadas para asignar 1 si alguna de las columnas tiene el valor 1
df['cochera_lineal'] = df.apply(lambda row: 1 if row['cochera_lineal'] == 1 or row['cochera_paralela'] == 1 or row['cochera_separadas'] == 1 else np.NaN, axis=1)

df.drop(columns=['cochera_paralela','cochera_separadas'], inplace=True)


In [35]:
df.rename(columns={'cochera_lineal': 'cochera_tiene'}, inplace=True)

In [36]:
columnas_actualizar = ['estado_excelente','cochera_tiene']
df[columnas_actualizar] = df[columnas_actualizar].apply(pd.to_numeric, errors="coerce").astype('category')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7113 entries, 0 to 7644
Data columns (total 96 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   antiguedad                7113 non-null   Int64   
 1   balneario                 7113 non-null   category
 2   baños                     7113 non-null   Int64   
 3   pisos                     7113 non-null   Int64   
 4   cocheras                  7113 non-null   Int64   
 5   descripcion               7113 non-null   object  
 6   dormitorios               7113 non-null   Int64   
 7   mascotas                  7113 non-null   category
 8   precio                    7113 non-null   float64 
 9   comercial                 762 non-null    category
 10  profesional               742 non-null    category
 11  latitud                   7113 non-null   float64 
 12  longitud                  7113 non-null   float64 
 13  area_constr               7113 non-null   float64 
 1

In [38]:
#Utilizando la biblioteca pickle de Python para generar un archivo de escritura binaria
pickle_out = open("../data/data.pkl","wb")
pickle.dump(df, pickle_out)
pickle_out.close()